In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

In [2]:
data = pd.read_csv('hotel_data.csv')

In [3]:
data.head(5)

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
0,Hardasji Ki Magri,Udaipur,India,2016-06-21,{{facility}},|Zion Home Stay is located in a city that sets...,1 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,78ddf880bd7937d384ff278cc5b39d6e
1,Near Nai Gaon,Udaipur,India,2016-06-21,{{facility}},| Araliayas Resorts is a 3 star hotel located ...,3 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,9f9f9cbb2f7df8089b63d5cdeb257944
2,Near Bagore Ki Haveli,Udaipur,India,2016-06-21,{{facility}},|A 2 star property is located at 24 km from Ma...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,NaN,makemytrip,NaN,NaN,b314bb7fa8bfb1ed306f517be21d729e
3,Dabok,Udaipur,India,2016-06-21,Airport Transfer|Car rental|Conference Hall|Cu...,|SNP House Airport Hotel And Restaurant is loa...,1 star,//imghtlak.mmtcdn.com/images/hotels/2014071815...,NaN,no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,`standard,/5,NaN,makemytrip,NaN,NaN,e6f5bb3c2d76a78d978b9ceb0e31ec56
4,East Udaipur,Udaipur,India,2016-06-21,{{facility}},| Hotel Pichola Haveli is situated in the beau...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,3.7,makemytrip,NaN,NaN,63072c301427b6ca450d31eea127bcf0


In [4]:
data.city.value_counts()

city
NewDelhiAndNCR    1163
Goa               1122
Mumbai             543
Jaipur             534
Bangalore          512
                  ... 
Kollur               1
Madla                1
Jeypore              1
Jispa                1
Haldia               1
Name: count, Length: 770, dtype: int64

In [5]:
array = ['Mumbai']

In [6]:
data = data.loc[data['city'].isin(array)]

In [7]:
data.head(5)

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
294,Charai,Mumbai,India,2016-08-28,Doctor on Call|Front desk|Laundry Service|Park...,"Nestled in Mumbai, a city with strong historic...",3,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Ac Superior Room|A/c Standard Rooms Double Occ...,NaN,NaN,makemytrip,Maharashtra,NaN,d78fae90ef2e1b5c2dfd547c61763a25
309,Andheri (East),Mumbai,India,2016-08-28,Air Conditioned|Airport Transfer|Conference Ha...,3 km from Chhatrapati Shivaji International Ai...,2,NaN,Bathroom Toiletries|Daily Newspaper|Kitchenett...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Executive|Deluxe,NaN,NaN,makemytrip,Maharasta,Location:3.2/5 | Hospitality:3.1/5 | Facilitie...,030865f741982beb373efddecdc6d6c3
321,Khar,Mumbai,India,2016-08-28,Airport/Rlwy Stn Transfer|Bar|Conference Hall|...,Location Hotel Royal Garden is situated on Juh...,3,NaN,Electronic Safe|Bathroom Toiletries|Daily News...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Superior Executive,NaN,NaN,makemytrip,Maharashtra,Location:4.5/5 | Hospitality:3.4/5 | Facilitie...,a1ced509350038775a7700ec67796bc2
334,Andheri (East),Mumbai,India,2016-08-28,24 Hour Check in-Icon|24 hour reception|24 hou...,City Guest House is a beautiful property locat...,2,NaN,Bathroom Toiletries|Hot/cold Water|Attached Ba...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Standard Room|Deluxe Room|Triple Deluxe Room|S...,NaN,NaN,makemytrip,Maharashtra,Location:4.3/5 | Hospitality:3.8/5 | Facilitie...,f2820ae7707668ed6906bb227921f720
1238,Andheri (East),Mumbai,NaN,2016-08-22,24 Hour Check in-Icon|24 hour reception|24 hou...,Sai Residency Hotel is situated in the City of...,2,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-22 22:10:53 +0000,2016-08-22 22:10:53 +0000,Deluxe Dbl Air Cooled,NaN,NaN,makemytrip,MAHARASHTRA,NaN,b4af24952027ffbcd85a91bb6fe23f5d


In [8]:
data = data.hotel_overview
data = data.dropna()

In [9]:
stop = set(stopwords.words('english'))
def stopwords_removal(data_point):
    data = [x for x in data_point.split() if x not in stop]
    return data

In [10]:
def clean_data(data):
    cleaned_data = []
    all_unique_words_in_each_description = []
    for entry in data:
        entry = re.sub(pattern='[^a-zA-Z]',repl=' ',string = entry)
        entry = re.sub(r'\b\w{0,1}\b', repl=' ',string = entry)
        entry = entry.lower()
        entry = stopwords_removal(entry)
        cleaned_data.append(entry)
        unique = list(set(entry))
        all_unique_words_in_each_description.extend(unique)
    return cleaned_data, all_unique_words_in_each_description

In [11]:
def unique_words(data):
    unique_words = set(all_unique_words_in_each_description)
    return unique_words, len(unique_words)

In [12]:
cleaned_data, all_unique_words_in_each_description = clean_data(data)
unique_words, length_of_unique_words = unique_words(all_unique_words_in_each_description)

In [13]:
cleaned_data[0]

['nestled',
 'mumbai',
 'city',
 'strong',
 'historical',
 'links',
 'wonderful',
 'british',
 'architecture',
 'museums',
 'beaches',
 'places',
 'worship',
 'true',
 'galaxy',
 'stars',
 'bollywood',
 'reigns',
 'supreme',
 'hotel',
 'divya',
 'international',
 'delightful',
 'leisure',
 'absolute',
 'blend',
 'service',
 'charm',
 'efficiency',
 'hotel',
 'offers',
 'facilities',
 'like',
 'front',
 'desk',
 'parking',
 'laundry',
 'doctor',
 'call',
 'many',
 'aims',
 'extend',
 'best',
 'possible',
 'hospitality',
 'experience',
 'revered',
 'customers',
 'hotel',
 'located',
 'distance',
 'workshop',
 'bus',
 'stop',
 'km',
 'chhatrapati',
 'shivaji',
 'international',
 'airport',
 'km',
 'chhatrapati',
 'shivaji',
 'terminus',
 'guests',
 'head',
 'prominent',
 'tourist',
 'attractions',
 'like',
 'sanjay',
 'gandhi',
 'national',
 'park',
 'kidzania',
 'mumbai',
 'haji',
 'ali',
 'mosque',
 'iskon',
 'temple',
 'shree',
 'siddhivinayak',
 'temple',
 'many',
 'shoppers',
 'fun',

In [14]:
length_of_unique_words

3395

In [15]:
def build_indices(unique_words):
    word_to_idx = {}
    idx_to_word = {}
    for i, word in enumerate(unique_words):
        word_to_idx[word] = i
        idx_to_word[i] = word
    return word_to_idx, idx_to_word

In [16]:
word_to_idx, idx_to_word = build_indices(unique_words)

In [17]:
def prepare_corpus(corpus, word_to_idx):
    
    sequences = []
    for line in corpus:
        tokens = line
        for i in range(1, len(tokens)):
            i_gram_sequence = tokens[:i+1]
            i_gram_sequence_ids = []
            
            for j, token in enumerate(i_gram_sequence):
                i_gram_sequence_ids.append(word_to_idx[token])
                
            sequences.append(i_gram_sequence_ids)
    
    return sequences

In [18]:
sequences = prepare_corpus(cleaned_data, word_to_idx)
max_sequence_len = max([len(x) for x in sequences])

In [19]:
print(sequences[0])
print(sequences[1])

[877, 1355]
[877, 1355, 2409]


In [20]:
print(idx_to_word[1647])
print(idx_to_word[867])
print(idx_to_word[1452])

said
stupendous
works


In [21]:
len(sequences)

51836

In [22]:
max_sequence_len

308

In [23]:
def build_input_data(sequences, max_sequence_len, length_of_unique_words):
    sequences = np.array(pad_sequences(sequences, maxlen = max_sequence_len, padding = 'pre'))
    X = sequences[:,:-1]
    y = sequences[:,-1]
    y = to_categorical(y, length_of_unique_words)
    return X, y

In [24]:
X, y = build_input_data(sequences, max_sequence_len, length_of_unique_words)
length_of_unique_words

3395

In [25]:
def create_model(max_sequence_len, length_of_unique_words):
    model = Sequential()
    model.add(Embedding(length_of_unique_words, 10, input_length=max_sequence_len - 1))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(length_of_unique_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [26]:
model = create_model(max_sequence_len, length_of_unique_words)

C:\Users\Leapfrog\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
# Explicitly build the model by specifying the input shape
model.build(input_shape=(None, max_sequence_len - 1))

# Now, try printing the summary again
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 307, 10)             │          33,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │          71,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3395)                │         437,955 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 543,073 (2.07 MB)

 Trainable params: 543,073 (2.07 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
len(X)

51836

In [29]:
model.fit(X, y, batch_size = 512, epochs=30)

Epoch 1/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 83s 776ms/step - loss: 7.3536
Epoch 2/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 82s 801ms/step - loss: 6.5620
Epoch 3/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 83s 817ms/step - loss: 6.4826
Epoch 4/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 89s 871ms/step - loss: 6.4270
Epoch 5/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 88s 867ms/step - loss: 6.3411
Epoch 6/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 92s 903ms/step - loss: 6.2617
Epoch 7/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 93s 908ms/step - loss: 6.1823
Epoch 8/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 91s 894ms/step - loss: 6.0763
Epoch 9/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 94s 924ms/step - loss: 5.9700
Epoch 10/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 102s 998ms/step - loss: 5.8808
Epoch 11/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - loss: 5.7959
Epoch 12/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 99s 965ms/step - loss: 5.6938
Epoch 13/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 144s 980ms/step - loss: 5.6092
Epoch 14/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 96s 943ms/step - loss: 5.5272
Epoch 15/30
102

In [35]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_text(seed_text, next_words, model, max_seq_len, word_to_idx, idx_to_word):
    for _ in range(next_words):
        # Assuming clean_data and prepare_corpus functions are defined elsewhere
        cleaned_data = clean_data([seed_text])
        sequences = prepare_corpus(cleaned_data[0], word_to_idx)
        
        # Pad the sequence
        sequences = pad_sequences([sequences[-1]], maxlen=max_seq_len - 1, padding='pre')
        
        # Predict the next word index
        predicted_probs = model.predict(sequences, verbose=0)
        predicted_index = np.argmax(predicted_probs, axis=1)[0]
        
        # Map the predicted index to the corresponding word
        output_word = idx_to_word.get(predicted_index, '')
        
        # Append the predicted word to the seed text
        seed_text += " " + output_word
        
    return seed_text.title()


In [36]:
print(generate_text("in Mumbai there we need", 30, model, max_sequence_len,word_to_idx, idx_to_word ))

In Mumbai There We Need Hotel Located Distance Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati


In [32]:
print(generate_text("Best Hotel Mumbai", 30, model, max_sequence_len,word_to_idx, idx_to_word ))

Best Hotel Mumbai Located Distance Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji


In [33]:
print(generate_text("The beauty of the city", 30, model, max_sequence_len,word_to_idx, idx_to_word ))

The Beauty Of The City Located Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International


In [37]:
model_structure = model.to_json()
with open("Output Files/text_generation_using_LSTM_10ephoc.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("Output Files/text_generation_using_LSTM_10ephoc.weights.h5")

### Summary
## Libraries Used

1. nltk: Natural Language Toolkit, used for text processing.
2. stopwords: To remove common stopwords from text data.
3. pandas: Used for data manipulation and analysis.
4. numpy: Used for numerical operations.
5. re: Regular expression operations for text cleaning.
6. keras:
Sequential: For creating a sequential neural network model.
Embedding: For converting words into dense vectors of fixed size.
LSTM: Long Short-Term Memory layer for handling sequential data.
Dropout: To prevent overfitting by randomly dropping neurons during training.
Dense: A fully connected layer.
7. tensorflow.keras.utils:
to_categorical: Converts a class vector (integers) to binary class matrix.

### Procedure Followed
1. Import Libraries:
The required libraries and modules were imported.
2. Load Data:
A dataset containing hotel reviews was loaded from a CSV file using pandas.
3. Initial Data Inspection:
The first few rows of the dataset were displayed to understand its structure.
The frequency of different cities in the dataset was analyzed.
4. Data Filtering:
The dataset was filtered to include only reviews from Mumbai.
5. Data Cleaning:
Missing values were dropped from the hotel_overview column.
A function was defined to remove stopwords from the text.
Another function was defined to clean the text by:
Removing non-alphabetic characters.
Converting text to lowercase.
Removing single-character words.
Removing stopwords.
The text data was cleaned using these functions, and unique words were extracted.
6. Building Word Indices:
Functions were defined to create mappings between words and their corresponding indices (and vice versa).
7. Preparing Sequences:
The cleaned text data was split into sequences of words.
Sequences were padded to ensure uniform length.
8. Prepare Input and Output Data:
The sequences were split into input (X) and output (y) components.
The output data (y) was converted to categorical format.
9. Model Creation:
A sequential model was defined with the following layers:
Embedding Layer: To convert input sequences to dense vectors.
LSTM Layer: To handle the sequential nature of the data.
Dropout Layer: To reduce overfitting.
Dense Layer: To output predictions with softmax activation.
The model was compiled with categorical_crossentropy loss and adam optimizer.
10. Model Summary:
The summary of the model was printed to verify its architecture.
11. Model Training:
The model was trained using the input data (X) and labels (y) for 100 epochs with a batch size of 512.
12. Text Generation:
A function was defined to generate text based on a seed input.
The function uses the trained model to predict the next word in the sequence iteratively.
Several examples of text generation were performed using different seed texts.
